In [25]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def get_displayed_date(date):
    url = f"https://tnagriculture.in/ARS/home/reservoir/{date}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    span = soup.find("span", class_="alert alert-danger")
    if span:
        text = span.get_text(strip=True)
        return text.replace("DATE :-", "").strip()
    return None

def find_first_real_date():
    current = datetime(2010, 1, 1)  # start from 2010
    end = datetime.now()            # today

    while current <= end:
        date_str = current.strftime("%Y-%m-%d")
        displayed = get_displayed_date(date_str)

        # If displayed date matches requested date, it's real
        if displayed == current.strftime("%d-%m-%Y"):
            return date_str

        current += timedelta(days=1)

    return None

first_real_date = find_first_real_date()
print("First date data is available:", first_real_date)


First date data is available: 2010-01-01


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

def get_data_for_date(date):
    url = f"https://tnagriculture.in/ARS/home/reservoir/{date}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    tds = soup.find_all("td")
    data = []

    for i in range(0, len(tds), 9):
        row = tds[i:i+9]
        if len(row) < 9:
            continue
        
        values = [x.get_text(strip=True) for x in row]
        data.append([
            date,
            values[0],  # Reservoir name
            values[1],  # Full Depth
            values[2],  # Full Capacity
            values[3],  # Current Year Level
            values[4],  # Current Year Storage
            values[5],  # Inflow
            values[6],  # Outflow
            values[7],  # Last Year Level
            values[8],  # Last Year Storage
        ])

    return data

# Generate dates for January 2024
start = datetime(2024, 1, 1)
end = datetime(2024, 1, 31)

all_data = []

current = start
while current <= end:
    date_str = current.strftime("%Y-%m-%d")
    all_data += get_data_for_date(date_str)
    current += timedelta(days=1)

# Create dataframe
df = pd.DataFrame(all_data, columns=[
    "Date", "Reservoirs", "Full Depth(Feet)", "Full Capacity(M.Cft.)",
    "Current Year Level(Feet)", "Current Year Storage(M.Cft.)",
    "Current Year Inflow(CuSecs)", "Current Year Outflow(CuSecs.)",
    "Last Year Level(Feet)", "Last Year Storage(M.Cft.)"
])

# Save to CSV
df.to_csv("January_2024_dam_data.csv", index=False)

print("Data saved successfully! Total rows:", len(df))


Data saved successfully! Total rows: 589


In [22]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def has_data(date):
    url = f"https://tnagriculture.in/ARS/home/reservoir/{date}"
    r = requests.get(url)
    if r.status_code != 200:
        return False

    soup = BeautifulSoup(r.text, "html.parser")
    tds = soup.find_all("td")
    return len(tds) > 0

def year_exists(year):
    dates = [f"{year}-01-01", f"{year}-06-01", f"{year}-12-01"]
    for d in dates:
        if has_data(d):
            return True
    return False

# 1) Find years with data
years = []
for y in range(2010, 2031):
    if year_exists(y):
        years.append(y)

print("Years with data:", years)

# 2) Create continuous ranges
ranges = []
start = years[0]
prev = years[0]

for y in years[1:]:
    if y == prev + 1:
        prev = y
    else:
        ranges.append((start, prev))
        start = y
        prev = y

ranges.append((start, prev))

print("Continuous ranges of data present:")
for r in ranges:
    print(r[0], "to", r[1])

# 3) Find last date available
def find_last_date():
    current = datetime(2030, 12, 31)  # start from far future
    end = datetime(2010, 1, 1)        # stop limit

    while current >= end:
        date_str = current.strftime("%Y-%m-%d")
        if has_data(date_str):
            return date_str
        current -= timedelta(days=1)

    return None

last_date = find_last_date()
print("Last date data is available:", last_date)


Years with data: [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]
Continuous ranges of data present:
2010 to 2030
Last date data is available: 2030-12-31


In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_data_for_date(date):
    url = f"https://tnagriculture.in/ARS/home/reservoir/{date}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    # Find all td elements in table
    tds = soup.find_all("td")

    # Each row has 9 columns
    data = []
    for i in range(0, len(tds), 9):
        row = tds[i:i+9]
        if len(row) < 9:
            continue
        
        # Extract text
        values = [x.get_text(strip=True) for x in row]
        
        # Add date + row
        data.append([
            date,
            values[0],  # Reservoir name
            values[1],  # Full Depth
            values[2],  # Full Capacity
            values[3],  # Current Year Level
            values[4],  # Current Year Storage
            values[5],  # Inflow
            values[6],  # Outflow
            values[7],  # Last Year Level
            values[8],  # Last Year Storage
        ])

    return data

# Test with one date
date = "2024-01-01"
data = get_data_for_date(date)

df = pd.DataFrame(data, columns=[
    "Date", "Reservoirs", "Full Depth(Feet)", "Full Capacity(M.Cft.)",
    "Current Year Level(Feet)", "Current Year Storage(M.Cft.)",
    "Current Year Inflow(CuSecs)", "Current Year Outflow(CuSecs.)",
    "Last Year Level(Feet)", "Last Year Storage(M.Cft.)"
])

df.head()

,Date,Reservoirs,Full Depth(Feet),Full Capacity(M.Cft.),Current Year Level(Feet),Current Year Storage(M.Cft.),Current Year Inflow(CuSecs),Current Year Outflow(CuSecs.),Last Year Level(Feet),Last Year Storage(M.Cft.)
0,2024-01-01,METTUR,120,93470,71.27,33801,807,252,118.79,91554
1,2024-01-01,Krishna Raja Sagar,124.8,49452,96.81,20412,661,975,118.94,41717
2,2024-01-01,Kabini,65,19516,54.29,13357,118,300,52.35,12352
3,2024-01-01,Harangi,129,8500,100.63,3371,154,100,98.01,3237
4,2024-01-01,Hemavathy,117,37103,86.66,14985,263,350,103.57,25618


In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import time

def get_data_for_date(date):
    url = f"https://tnagriculture.in/ARS/home/reservoir/{date}"
    r = requests.get(url)

    # If website is down or page not found
    if r.status_code != 200:
        return []

    soup = BeautifulSoup(r.text, "html.parser")
    tds = soup.find_all("td")

    data = []
    for i in range(0, len(tds), 9):
        row = tds[i:i+9]
        if len(row) < 9:
            continue

        values = [x.get_text(strip=True) for x in row]
        data.append([
            date,
            values[0],  # Reservoir name
            values[1],  # Full Depth
            values[2],  # Full Capacity
            values[3],  # Current Year Level
            values[4],  # Current Year Storage
            values[5],  # Inflow
            values[6],  # Outflow
            values[7],  # Last Year Level
            values[8],  # Last Year Storage
        ])

    return data

# ------------------ MAIN SCRAPER ------------------

start_date = datetime(2022, 5, 26)
end_date = datetime(2025, 12, 31)

all_data = []
current_date = start_date

while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    daily_data = get_data_for_date(date_str)

    if daily_data:
        all_data.extend(daily_data)
        print(f"Data added for: {date_str}  -> {len(daily_data)} rows")

    # Move to next day
    current_date += timedelta(days=1)

    # Sleep to avoid blocking (important for websites)
    time.sleep(0.5)

# ------------------ SAVE TO CSV ------------------

df = pd.DataFrame(all_data, columns=[
    "Date", "Reservoirs", "Full Depth(Feet)", "Full Capacity(M.Cft.)",
    "Current Year Level(Feet)", "Current Year Storage(M.Cft.)",
    "Current Year Inflow(CuSecs)", "Current Year Outflow(CuSecs.)",
    "Last Year Level(Feet)", "Last Year Storage(M.Cft.)"
])

df.to_csv("dam_data_2015_2025.csv", index=False)
print("✅ Saved as dam_data_2015_2025.csv")
print("Total rows:", len(df))

Data added for: 2022-05-26  -> 19 rows
Data added for: 2022-05-27  -> 19 rows
Data added for: 2022-05-28  -> 19 rows
Data added for: 2022-05-29  -> 19 rows
Data added for: 2022-05-30  -> 19 rows
Data added for: 2022-05-31  -> 19 rows
Data added for: 2022-06-01  -> 19 rows
Data added for: 2022-06-02  -> 19 rows
Data added for: 2022-06-03  -> 19 rows
Data added for: 2022-06-04  -> 19 rows
Data added for: 2022-06-05  -> 19 rows
Data added for: 2022-06-06  -> 19 rows
Data added for: 2022-06-07  -> 19 rows
Data added for: 2022-06-08  -> 19 rows
Data added for: 2022-06-09  -> 19 rows
Data added for: 2022-06-10  -> 19 rows
Data added for: 2022-06-11  -> 19 rows
Data added for: 2022-06-12  -> 19 rows
Data added for: 2022-06-13  -> 19 rows
Data added for: 2022-06-14  -> 19 rows
Data added for: 2022-06-15  -> 19 rows
Data added for: 2022-06-16  -> 19 rows
Data added for: 2022-06-17  -> 19 rows
Data added for: 2022-06-18  -> 19 rows
Data added for: 2022-06-19  -> 19 rows
Data added for: 2022-06-2

In [30]:
# while getting ip blokage in web scrapping i stored the generated data here.
pd.DataFrame(
    all_data,
    columns=[
        "Date", "Reservoirs", "Full Depth", "Full Capacity",
        "Current Level", "Current Storage",
        "Inflow", "Outflow",
        "Last Year Level", "Last Year Storage"
    ]
).to_csv("dam_data_emergency_save.csv", index=False)

# also there will be anotehr file which is generated after scrapping the remaning data.

In [44]:
# I am combining both csv generated befored and after emergency.
import pandas as pd

# Load both CSV files
df1 = pd.read_csv("dam_data_emergency_save.csv")
df2 = pd.read_csv("dam_data_2015_2025.csv")

print("File 1 rows:", len(df1))
print("File 2 rows:", len(df2))

# Combine both datasets
combined_df = pd.concat([df1, df2], ignore_index=True)

# Remove exact duplicate rows
combined_df.drop_duplicates(inplace=True)

# If you have a Date column, sort by it
if "Date" in combined_df.columns:
    combined_df["Date"] = pd.to_datetime(combined_df["Date"], errors="coerce")
    combined_df.sort_values("Date", inplace=True)

# Reset index after cleaning
combined_df.reset_index(drop=True, inplace=True)

# Save final merged file
final_file = "dam_data_2015_2025_with_error.csv"
combined_df.to_csv(final_file, index=False)

print("Final rows:", len(combined_df))
print("Saved as:", final_file)

File 1 rows: 51287
File 2 rows: 25042
Final rows: 76279
Saved as: dam_data_2015_2025_final.csv


## Success in combination with a problem.
1. After doing all the efforts for the extracting data I successfully scraped all the 76279 rows from the government website after 4000 hits on the websites.
2. The problem is both files have differnt column names. So, they are not combining properly they are making differnt columns after row number 51287. It means we are getting 20 columsn and its a big problem.
## My solution on the problem
1. Standardizing the column names in the both files.
2. Combining both files with simple column names
3. studying the columns units and columns itself for further process.

In [34]:
# standerdizing the columns
standard_columns = [
    "date",
    "dam_name",
    "full_depth",
    "full_capacity",
    "current_level",
    "current_storage",
    "current_inflow",
    "current_outflow",
    "Last Year Level(Feet)",
    "Last Year Storage(M.Cft.)"
]

In [38]:
# writing a function to standerdize the files columsn
def standerdize_columns(input_file,output_file):
    df=pd.read_csv(input_file)

    if len(df.columns)!=len(standard_columns):
        raise ValueError(f'column count missmatch in {input_file}')

    df.columns=standard_columns
    df.to_csv(output_file,index=False) # while merging both files we add index

    print(f'Standerdized and saved: {output_file}')

In [40]:
# my first file which was saved in emergency of ip adderss block is standardized with the columns
standerdize_columns("dam_data_emergency_save.csv","dam_data_emergency_standard.csv")

Standerdized and saved: dam_data_emergency_standard.csv


In [43]:
# my second file after emergency standerdized columns
standerdize_columns("dam_data_2015_2025.csv","dam_data_2015_2025_standard.csv")

Standerdized and saved: dam_data_2015_2025_standard.csv


In [53]:
# combining the standerdized columns files
# loding both csv files
df1=pd.read_csv("dam_data_emergency_standard.csv")
df2=pd.read_csv("dam_data_2015_2025_standard.csv")

print("length of rows in file 1 : {}".format(len(df1)))
print("length of rows in file 2 : {}".format(len(df2)))

# combining both files
combined_df=pd.concat([df1,df2],ignore_index=True)

# removing the duplicate rows
combined_df.drop_duplicates(inplace=True)

# sorting date column
if "date" in combined_df.columns:
    combined_df["date"]=pd.to_datetime(combined_df["date"],errors="coerce")
    combined_df.sort_values("date",inplace=True)

# Reseting the index
combined_df.reset_index(drop=True,inplace=True) # drop=True for the removing old index.

# giving name to file and converting it to csv
file_name="dam_data_final.csv"   # alwaays give .csv other wise you get normal data with commmas
combined_df.to_csv(file_name,index=False) # index=False to maek data more clear. at last before sending data to ml we clear the index

print("Final rows after combination : ", len(combined_df))
print("Final combined file saved as : ",file_name)

length of rows in file 1 : 51287
length of rows in file 2 : 25042
Final rows after combination :  76279
Final combined file saved as :  dam_data_final.csv
